In [1]:
import torch
import torch.nn as nn
import utils
import BuildingBlocks
vocab = utils.Tokenizer.readVocabulary(r"F:\RetentionTimeProject\SimpleVocab.csv")

[WARNING 04-19 10:54:10] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


In [2]:
class ResNetBlockXavierNormal(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size,   bias):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        
        
        self.bias = bias

        self.embedding = nn.Embedding(len(vocab), 64, 0)
        self.conv1 = nn.Conv1d(in_channels, out_channels,bias=bias, kernel_size=kernel_size, padding=1, stride=1, dilation=1)
        self.conv2 = nn.Conv1d(out_channels, out_channels,bias=bias, kernel_size=kernel_size, padding=1, stride=1, dilation=1)
        self.batchNorm1 = nn.BatchNorm1d(out_channels)
        self.batchNorm2 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()

        #init weights
        nn.init.xavier_normal_(self.conv1.weight)
        nn.init.xavier_normal_(self.conv2.weight)

    def forward(self, x):
        x = self.embedding(x)
        residual = x
        x = self.conv1(x)
        # x = utils.padTensorBatchAfterCNN(x, self.in_channels)
        x = self.batchNorm1(x)
        x = self.relu(x)
        x = self.conv2(x)
        # x = utils.padTensorBatchAfterCNN(x, self.in_channels)
        x = self.batchNorm2(x)
        #x = utils.padTensorBatchAfterCNN(x, self.in_channels)
        x += residual
        x = self.relu(x)
        return x
    
class OutputLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.linear = nn.Linear(in_channels, 64*(self.in_channels//64))
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(out_channels, 1)
        nn.init.xavier_normal_(self.linear.weight)
    
    def forward(self, x):
        x = x.view(self.in_channels//64, 64*(self.in_channels//64))
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

In [33]:
class ResNetBlockXavierNormal(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1, stride=1, dilation=1):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, dilation=dilation)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride, dilation=dilation)
        self.batchNorm1 = nn.BatchNorm1d(out_channels)
        self.batchNorm2 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()

        self.double()

        #init weights
        nn.init.xavier_normal_(self.conv1.weight)
        nn.init.xavier_normal_(self.conv2.weight)

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        # x = utils.padTensorBatchAfterCNN(x, self.in_channels)
        x = self.batchNorm1(x)
        x = self.relu(x)
        x = self.conv2(x)
        # x = utils.padTensorBatchAfterCNN(x, self.in_channels)
        x = self.batchNorm2(x)
        #x = utils.padTensorBatchAfterCNN(x, self.in_channels)
        x += residual
        x = self.relu(x)
        return x
    
class ResNetiRT(nn.Module):
    def __init__(self, numBlocks, batchSize):
        super().__init__()
        self.numBlocks = numBlocks
        self.batchSize = batchSize
        self.embedding = nn.Embedding(len(vocab), 64, 0)
        self.blocks = nn.ModuleList()
        # self.blocks.append(BuildingBlocks.DimSwap())
        for i in range(numBlocks):
            self.blocks.append(ResNetBlockXavierNormal(64, 64))
        self.outputLayer = nn.Linear(64*64, 1)
        self.double()
    
    def forward(self, x):
        x = self.embedding(x)
        for block in self.blocks:
            x = block(x)
        x = x.flatten(1)
        x = self.outputLayer(x)
        return x
    
model = ResNetiRT(10, 1)
model(torch.zeros(32, 64, dtype=torch.int64)).shape

torch.Size([32, 1])

In [13]:
for layer in model.modules():
    print(layer)

ResNetBlockXavierNormal(
  (embedding): Embedding(223, 64, padding_idx=0)
  (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (batchNorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchNorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)
Embedding(223, 64, padding_idx=0)
Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU()
